In [ ]:
# Andrea Ghezzi, Adrian Buerger, University of Freiburg, 2022

In [ ]:
import copy
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
# mpl.rcParams["backend"] = 'agg'

In [ ]:
golden = (1 + 5 ** 0.5) / 2
def latexify():
    params = {
        # 'backend': 'ps',
        "text.latex.preamble": r"\usepackage{gensymb} \usepackage{amsmath} \usepackage{amssymb}",
        "lines.linewidth": 1,
        "figure.figsize": (3.5, 4),
        "axes.labelsize": 7,
        "axes.titlesize": 8,
        "legend.fontsize": 8,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
        "text.usetex": True,
        "font.family": "serif",
    }
    mpl.rcParams.update(params)

In [ ]:
WEIGHT = 'ID' #CIA, ID
STRATEGY = 'STD-ext' #STD, RES

In [ ]:
if (WEIGHT=='CIA') & (STRATEGY=='STD'):
    CASE_NUMBER = 1
elif (WEIGHT=='CIA') & (STRATEGY=='RES'):
    CASE_NUMBER = 2
elif (WEIGHT=='ID') & (STRATEGY=='STD'):
    CASE_NUMBER = 3
elif (WEIGHT=='ID') & (STRATEGY=='RES'):
    CASE_NUMBER = 4
elif (WEIGHT=='CIA') & (STRATEGY=='STD-ext'):
    CASE_NUMBER = 5
elif (WEIGHT=='ID') & (STRATEGY=='STD-ext'):
    CASE_NUMBER = 6

RESULTS_DIR = f'/Users/aghezzi/Documents/results-voronoi-veela/{CASE_NUMBER}-{WEIGHT.lower()}-{STRATEGY.lower()}-json/'
TOTAL_ITER = max(set([int(st.split('.')[0].split('_')[-1]) for st in os.listdir(RESULTS_DIR) if (st.split('.')[0].split('_')[-1] != 'rel') & (st!='.DS_Store')])) + 1

if CASE_NUMBER == 5:
    STRATEGY = 'STD'
if CASE_NUMBER == 6:
    STRATEGY = 'STD'
MIQP_NAME = lambda iter_nr: f'binapprox_miqp_{WEIGHT}_{STRATEGY}_iter_{iter_nr}.json'
NLP_NAME = lambda iter_nr: f'nlpsolver_bin_miqp_{WEIGHT}_{STRATEGY}_iter_{iter_nr}.json'

In [ ]:
with open(RESULTS_DIR + 'nlpsolver_rel.json', 'rb') as f:
        nlpsolver_rel = json.load(f)

In [ ]:
iter_nr = 0

In [ ]:
with open(RESULTS_DIR + MIQP_NAME(iter_nr=iter_nr), "rb") as f:
        nlpsolver_miqp = json.load(f)

with open(RESULTS_DIR + NLP_NAME(iter_nr=iter_nr), "rb") as f:
        nlpsolver_nlp_bin = json.load(f)

In [ ]:
nlpsolver_miqp['data'].keys()

In [ ]:
nlpsolver_nlp_bin['data'].keys()

### Plot NLP objectives and computation time

In [ ]:
with open(RESULTS_DIR + "nlpsolver_rel.json", "rb") as f:
            nlpsolver_rel = json.load(f)

In [ ]:
nlp_objective_list = []
nlp_wall_time_list = []
for iter_nr in range(TOTAL_ITER):
    with open(RESULTS_DIR + NLP_NAME(iter_nr=iter_nr), "rb") as f:
            nlpsolver_nlp_bin = json.load(f)
    nlp_objective_list.append(nlpsolver_nlp_bin['data']['nlp_objective_value'])
    nlp_wall_time_list.append(nlpsolver_nlp_bin['solver_wall_time'])

miqp_wall_time_list = []
for iter_nr in range(TOTAL_ITER):
    with open(RESULTS_DIR + MIQP_NAME(iter_nr=iter_nr), "rb") as f:
            nlpsolver_miqp = json.load(f)
    miqp_wall_time_list.append(nlpsolver_miqp['solver_wall_time'])


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

x_axis_list = np.arange(0, TOTAL_ITER)

ax[0].set_title(f"Simulation with strategy={STRATEGY} and weight={WEIGHT}", pad=12.)

ax[0].axhline(
    nlpsolver_rel['data']['nlp_objective_value'],
    color="C1",
    linestyle="dashed",
    label="NLP_rel",
)
ax[0].bar(
    x=x_axis_list,
    height=nlp_objective_list, color="C0", label="NLP_bin",
    alpha=0.5
    )

ax[0].annotate(
    r"NLP_bin/NLP_rel="+"%.2f" %(nlp_objective_list[0]/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(0, nlp_objective_list[0]*1.2)
    )
# ax[0].scatter(0, obj_values_bin[0], color="C0", marker="o", label="NLP_bin_init")
# ax[0].scatter(iter_min_obj, min_obj, color="C0", marker="x", label="NLP_bin_min")
ax[0].legend(loc="upper center")
ax[0].set_ylabel("Ipopt objective value")

ax0a = ax[0].twinx()
ax0a.plot(
    x_axis_list,
    np.array(nlp_objective_list) / nlpsolver_rel['data']['nlp_objective_value'],
    linestyle="dotted",
    color="C7",
    label="NLP_bin/NLP_rel",
    marker='o'
)
ax0a.legend(loc="upper right")
# ax0a.set_yscale("log")
ax0a.set_ylim(1.05, 2)
ax0a.set_ylabel("Relation NLP_bin/NLP_rel (log scale)")

ax[0].scatter(
    nlp_objective_list.index(np.min(nlp_objective_list)),
    np.min(nlp_objective_list),
    color='r', marker='*', s=100
    )
ax[0].annotate(
    r"NLP_bin/NLP_rel="+"%.2f" %(np.min(nlp_objective_list)/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(
        nlp_objective_list.index(np.min(nlp_objective_list)),
        np.min(nlp_objective_list)*1.2)
    )


ax[1].plot(x_axis_list, miqp_wall_time_list, color="C0", label="gurobi")
ax[1].plot(x_axis_list, nlp_wall_time_list, color="C1", label="ipopt")
ax[1].set_yscale("log")
ax[1].grid(visible=True, which="both", axis="y", linestyle="dotted")
ax[1].legend(loc="best")
ax[1].set_ylabel("Runtime (s)")
ax[1].set_xlabel("Iteration")

ax[1].set_xticks(x_axis_list)


# ax[1].set_xlim(1, len(nlp_objective_list))

for ax_k in ax:
    for pos in ["top", "right"]:
        ax_k.spines[pos].set_visible(False)

ax0a.spines["top"].set_visible(False)

# plt.savefig(PLOTFILE_NAME, format="png", bbox_inches="tight")

In [ ]:
latexify()
fig, ax = plt.subplots(1, 2, figsize=(8, 4/golden), gridspec_kw={'width_ratios': [2.5, 1]})

x_axis_list = np.arange(0, TOTAL_ITER)

# ax[0].set_title(f"Simulation with strategy={STRATEGY} and weight={WEIGHT}", pad=12.)

ax[0].bar(
    x=x_axis_list,
    height=nlp_objective_list, color="C0", label=r"$\mathrm{NLP}_\mathrm{bin}$",
    alpha=0.5
    )
ax[0].axhline(
    nlpsolver_rel['data']['nlp_objective_value'],
    color="C1",
    linestyle="dashed",
    label=r"$\mathrm{NLP}_\mathrm{rel}$",
)
ax[0].scatter(
    0,
    nlp_objective_list[0],
    color='r', marker='.', s=25
    )
ax[0].annotate(
    r"$\frac{\mathrm{NLP}_\mathrm{bin}}{\mathrm{NLP}_\mathrm{rel}}=$"+r"$%.2f$" %(nlp_objective_list[0]/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(-1.4, nlp_objective_list[0]*1.15)
    )
# ax[0].scatter(0, obj_values_bin[0], color="C0", marker="o", label="NLP_bin_init")
# ax[0].scatter(iter_min_obj, min_obj, color="C0", marker="x", label="NLP_bin_min")
ax[0].legend(loc="upper center")
ax[0].set_ylabel("Objective value")
ax[0].set_ylim(1500, nlpsolver_rel['data']['nlp_objective_value']*2.5)
ax[0].set_xlim(-1.5, len(nlp_objective_list))


# ax0a = ax[0].twinx()
# ax0a.plot(
#     x_axis_list,
#     np.array(nlp_objective_list) / nlpsolver_rel['data']['nlp_objective_value'],
#     linestyle="dotted",
#     color="C7",
#     label="NLP_bin/NLP_rel",
#     marker='o'
# )
# ax0a.legend(loc="upper right")
# # ax0a.set_yscale("log")
# ax0a.set_ylim(1.05, 2)
# ax0a.set_ylabel("Relation NLP_bin/NLP_rel (log scale)")

ax[0].scatter(
    nlp_objective_list.index(np.min(nlp_objective_list)),
    np.min(nlp_objective_list),
    color='r', marker='*', s=25
    )
ax[0].annotate(
    r"$\frac{\mathrm{NLP}_\mathrm{bin}}{\mathrm{NLP}_\mathrm{rel}}=$"+r"$%.2f$" %(np.min(nlp_objective_list)/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(
        nlp_objective_list.index(np.min(nlp_objective_list))-1.5,
        np.min(nlp_objective_list)*1.2)
    )
ax[0].set_xlabel("Iteration")
xtick_labels = -1 * np.ones(len(x_axis_list))
xtick_labels[::2] = np.array(x_axis_list)[::2]
xtick_labels = [None if elm==-1 else int(elm) for elm in xtick_labels]
ax[0].set_xticks(np.array(x_axis_list), xtick_labels)

# ax[0].set_xticks(np.array(x_axis_list), labels=xtick_labels.astype(int))
# ax[0].set_xticks(np.array(x_axis_list), np.array(x_axis_list)[::2])

# ax[0].set_xticklabels(xtick_labels)


# ax[1].plot(x_axis_list, np.array(miqp_wall_time_list)/3600, color="C0", label="gurobi")
# ax[1].plot(x_axis_list, np.array(nlp_wall_time_list)/3600, color="C1", label="ipopt")
ax[1].plot(x_axis_list, np.cumsum(np.array(miqp_wall_time_list)/3600),
    marker='.', markersize=5, color="C0", alpha=0.7, label="MIQP: gurobi")
# ax[1].set_yscale("log")
ax[1].grid(visible=True, which="both", axis="y", linestyle="dotted")
# ax[1].legend(loc="best")
ax[1].set_ylabel("Cumulative runtime (hour)")
ax[1].set_xlabel("Iteration")
ax[1].set_xticks(x_axis_list[::5])
ax1a = ax[1].twinx()
ax1a.plot(x_axis_list, np.cumsum(np.array(nlp_wall_time_list)/60),
    marker='.', markersize=5, color="C1", alpha=0.7, label=r"$\mathrm{NLP}_\mathrm{bin}$: ipopt")
ax1a.set_ylabel("Cumulative runtime (min)")
for pos in ["top"]:
    ax1a.spines[pos].set_visible(False)

h, l = ax[1].get_legend_handles_labels()
h1, l1 = ax1a.get_legend_handles_labels()
ax[1].legend(h + h1, l + l1, loc='best')
ax[1].set_xlim(1, len(nlp_objective_list))

for ax_k in ax:
    for pos in ["top", "right"]:
        ax_k.spines[pos].set_visible(False)

ax1a.spines["top"].set_visible(False)

plt.tight_layout()
# fig.savefig("../poster-elox/figures/alg_iter_comparison.pdf", bbox_inches='tight', pad_inches=0.01)
plt.show();

In [ ]:
latexify()
fig, ax = plt.subplots(2, 1, figsize=(3.2, 2.7)) #, gridspec_kw={'height_ratios': [1, 2]})

x_axis_list = np.arange(0, TOTAL_ITER)

ax[0].bar(
    x=x_axis_list,
    height=nlp_objective_list, color="C0", label=r"$J_k$",
    alpha=0.5
    )
ax[0].axhline(
    nlpsolver_rel['data']['nlp_objective_value'],
    color="C1",
    linestyle="dashed",
    label=r"$J^\circ$",
)
ax[0].scatter(
    0,
    nlp_objective_list[0],
    color='r', marker='.', s=25
    )
ax[0].annotate(
    r"$\frac{J_0}{J^\circ}=$"+r"$%.2f$" %(nlp_objective_list[0]/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(-1.4, nlp_objective_list[0]*1.15)
    )
ax[0].legend(loc="upper center", framealpha=0.0, markerscale=0.1)
ax[0].set_ylabel(r"Objective $(\cdot 10^3)$", fontsize=7)
ax[0].set_ylim(1500, nlpsolver_rel['data']['nlp_objective_value']*2.5)
ax[0].set_yticks(np.array([2000,3000,4000]), [2, 3, 4])

ax[0].scatter(
    nlp_objective_list.index(np.min(nlp_objective_list)),
    np.min(nlp_objective_list),
    color='r', marker='*', s=25
    )
ax[0].annotate(
    r"$\frac{J_{23}}{J^\circ}=$"+r"$%.2f$" %(np.min(nlp_objective_list)/nlpsolver_rel['data']['nlp_objective_value']),
    xy=(
        nlp_objective_list.index(np.min(nlp_objective_list))-1.5,
        np.min(nlp_objective_list)*1.2)
    )
# ax[0].set_xlabel("Iteration")
ax[0].set_xlim(-1.5, len(nlp_objective_list))
xtick_labels = -1 * np.ones(len(x_axis_list))
xtick_labels[::2] = np.array(x_axis_list)[::2]
xtick_labels = [None if elm==-1 else int(elm) for elm in xtick_labels]
ax[0].set_xticks(np.array(x_axis_list), xtick_labels, size=7)

# ax[1].set_title("Cumulative runtime", pad=-0.05)
ax[1].plot(x_axis_list, np.cumsum(np.array(miqp_wall_time_list)/3600),
    marker='.', markersize=5, color="C0", alpha=0.7, label=r"$\mathcal{P}_\mathrm{GN}$: Gurobi")
ax[1].grid(visible=True, which="both", axis="y", linestyle="dotted")
ax[1].set_ylabel("Runtime (h)", fontsize=7)
ax[1].set_yticks(np.array([50, 100, 150]), np.array([50, 100, 150]))
ax[1].set_xlabel("Iteration")
ax[1].set_xticks(x_axis_list[::5])
ax1a = ax[1].twinx()
ax1a.plot(x_axis_list, np.cumsum(np.array(nlp_wall_time_list)/60),
    marker='^', markersize=3, color="C1", alpha=0.7, label=r"$\mathcal{P}$: Ipopt")
ax1a.set_ylabel("Runtime (min)", fontsize=7)
for pos in ["top"]:
    ax1a.spines[pos].set_visible(False)

h, l = ax[1].get_legend_handles_labels()
h1, l1 = ax1a.get_legend_handles_labels()
ax[1].legend(h + h1, l + l1, loc='best', framealpha=0.0)
ax[1].set_xlim(-1.5, len(nlp_objective_list))
my_xlabels = [elm if elm%5 == 0 else "" for elm in x_axis_list]
ax[1].set_xticks(np.array(x_axis_list), my_xlabels, size=7)

for ax_k in ax:
    ax_k.get_yaxis().set_label_coords(-0.11, 0.5)
    for pos in ["top", "right"]:
        ax_k.spines[pos].set_visible(False)

ax1a.spines["top"].set_visible(False)

ax1a.spines['left'].set_color("C0")
ax1a.spines['right'].set_color("C1")
ax[1].tick_params(axis='y', colors='C0')
ax1a.tick_params(axis='y', colors='C1')
ax[1].yaxis.label.set_color("C0")
ax1a.yaxis.label.set_color("C1")

fig.subplots_adjust(hspace=0.27)
# plt.tight_layout()
fig.savefig("latex/figures/alg_iter_comparison.pdf", bbox_inches='tight', pad_inches=0.01)
plt.show()

In [ ]:
ax[0].get_yticklabels()[2]

### Plot state trajectories
Iteration 0 vs. best iteration

In [ ]:
optimal_iter = nlp_objective_list.index(np.min(nlp_objective_list))
time_grid = nlpsolver_rel['time_points']
time_grid = pd.to_datetime(time_grid).hour + pd.to_datetime(time_grid).minute * 1/60
# time_grid = np.linspace(0, 24, 24*4+1)
time_grid = np.concatenate([np.array(time_grid[-9:]), np.array(time_grid[1:-9])])
# time_grid = np.concatenate([np.array(time_grid[-9:]), np.array(time_grid[1:-9]), np.array([0.])])
# time_grid

In [ ]:
time_grid.shape

In [ ]:
with open(RESULTS_DIR + NLP_NAME(iter_nr=optimal_iter), "rb") as f:
        best_nlpsolver_nlp_bin = json.load(f)
with open(RESULTS_DIR + NLP_NAME(iter_nr=0), "rb") as f:
        init_nlpsolver_nlp_bin = json.load(f)

In [ ]:
latexify()
fig, ax = plt.subplots(1, 3, figsize=(8,3/golden))

rel_c_data = np.array(nlpsolver_rel["data"]["c_data"])[:-1, :]

ax[0].plot(time_grid, rel_c_data[:,1]/1e3, color = "#d62728ff", label = "$I_\mathsf{fpsc}$")
ax[0].plot(time_grid, rel_c_data[:,2]/1e3, color = "#8c0e6ca0", label = "$I_\mathsf{vtsc}$")
ax0a = ax[0].twinx()
ax0a.plot(time_grid, rel_c_data[:,4]/1e3, color = "C2", label = "$P_\mathsf{pv,kWp}$")

ax[0].set_ylim(0,1)
ax[0].set_ylabel("Irrad. (kW/m²)")

ax0a.set_ylim(0,1)
ax0a.set_ylabel("Power kW")
# ax0a.legend(loc="upper right", framealpha=0.0)
l , h = ax[0].get_legend_handles_labels()
l1, h1, = ax0a.get_legend_handles_labels()
ax[0].legend(l+l1, h+h1, loc="upper right", framealpha=0.0)

ax[1].plot(time_grid, rel_c_data[:,0], color = "C0", label = "$T_\mathsf{amb}$")
#ax[0].set_ylim(0,950)
ax[1].set_ylabel("Temp. (°C)", labelpad=12)
ax[1].set_ylim(10, 30)
# ax[1].legend(loc="lower center", framealpha=0.0,  bbox_to_anchor=(0.35, 0.05))

ax1a = ax[1].twinx()
ax1a.plot(time_grid, rel_c_data[:,3]/1e3, color = "C1", label = "$\dot{Q}_\mathsf{lc}$")
ax1a.set_ylabel("Load (kW)")
ax1a.set_ylim(0, 9)
# ax1a.legend(loc="upper right", framealpha=0.0)
l , h = ax[1].get_legend_handles_labels()
l1, h1, = ax1a.get_legend_handles_labels()
ax[1].legend(l+l1, h+h1, loc="lower center", framealpha=0.0)

ax[2].plot(time_grid, rel_c_data[:,5], color = "C0", label = "$p_\mathsf{g}$")
ax[2].set_ylim(0,20)
ax[2].set_ylabel("Price (ct/kWh)")
ax[2].legend(loc="upper right", framealpha=0.0)


for ax_k in ax:
    ax_k.set_xlim(time_grid[0], time_grid[-1])
    ax_k.set_xlabel("Time (h)")
    ax_k.spines["top"].set_visible(False)
    ax_k.spines["right"].set_visible(False)
ax0a.spines["top"].set_visible(False)
ax1a.spines["top"].set_visible(False)

plt.tight_layout()
# fig.savefig("../poster-elox/figures/ambient_condition_energy_system.pdf", bbox_inches='tight', pad_inches=0.01)
plt.show();

In [ ]:
latexify()
fig, ax = plt.subplots(3,1, figsize=(4,3), sharex=True)

rel_c_data = np.array(nlpsolver_rel["data"]["c_data"])[:-1, :]

ax[0].plot(time_grid, rel_c_data[:,1]/1e3, color = "#d62728ff", label = "$I_\mathsf{fpsc}$")
ax[0].plot(time_grid, rel_c_data[:,2]/1e3, color = "#8c0e6ca0", label = "$I_\mathsf{vtsc}$")
ax0a = ax[0].twinx()
ax0a.plot(time_grid, rel_c_data[:,4]/1e3, color = "C2", label = "$P_\mathsf{pv,kWp}$")

ax[0].set_ylim(0,1)
ax[0].set_ylabel("Irrad. (kW/m²)")

ax0a.set_ylim(0,1)
ax0a.set_ylabel("Power kW")
# ax0a.legend(loc="upper right", framealpha=0.0)
l , h = ax[0].get_legend_handles_labels()
l1, h1, = ax0a.get_legend_handles_labels()
ax[0].legend(l+l1, h+h1, loc="upper right", framealpha=0.0)

ax[1].plot(time_grid, rel_c_data[:,0], color = "C0", label = "$T_\mathsf{amb}$")
#ax[0].set_ylim(0,950)
ax[1].set_ylabel("Temp. (°C)", labelpad=12)
ax[1].set_ylim(10, 30)
# ax[1].legend(loc="lower center", framealpha=0.0,  bbox_to_anchor=(0.35, 0.05))

ax1a = ax[1].twinx()
ax1a.plot(time_grid, rel_c_data[:,3]/1e3, color = "C1", label = "$\dot{Q}_\mathsf{lc}$")
ax1a.set_ylabel("Load (kW)")
ax1a.set_ylim(0, 9)
# ax1a.legend(loc="upper right", framealpha=0.0)
l , h = ax[1].get_legend_handles_labels()
l1, h1, = ax1a.get_legend_handles_labels()
ax[1].legend(l+l1, h+h1, loc="lower center", framealpha=0.0)

ax[2].plot(time_grid, rel_c_data[:,5], color = "C0", label = "$p_\mathsf{g}$")
ax[2].set_ylim(0,20)
ax[2].set_ylabel("Price (ct/kWh)")
ax[2].legend(loc="upper right", framealpha=0.0)


for ax_k in ax:
    ax_k.set_xlim(time_grid[0], time_grid[-1])
    ax_k.spines["top"].set_visible(False)
    ax_k.spines["right"].set_visible(False)
    ax_k.get_yaxis().set_label_coords(-0.1,0.5)
ax[2].set_xlabel("Time (h)")
ax0a.spines["top"].set_visible(False)
ax1a.spines["top"].set_visible(False)
ax0a.get_yaxis().set_label_coords(1.1,0.5)
ax1a.get_yaxis().set_label_coords(1.1,0.5)
# plt.tight_layout()
# fig.savefig("latex/figures/ambient_condition_energy_system.pdf", bbox_inches='tight', pad_inches=0.01)
plt.show();

In [ ]:
latexify()
fig, ax = plt.subplots(5, 2, sharex=True, figsize=(7.5,8/golden))

best_x_data = np.array(best_nlpsolver_nlp_bin["data"]["x_data"])[:-1, :]
init_x_data = np.array(init_nlpsolver_nlp_bin["data"]["x_data"])[:-1, :]
rel_x_data = np.array(nlpsolver_rel["data"]["x_data"])[:-1, :]
best_b_data = np.array(best_nlpsolver_nlp_bin["data"]["b_data"])[:-1, :]
init_b_data = np.array(init_nlpsolver_nlp_bin["data"]["b_data"])[:-1, :]
rel_b_data = np.array(nlpsolver_rel["data"]["b_data"])[:-1, :]
best_u_data = np.array(best_nlpsolver_nlp_bin["data"]["u_data"])[:-1, :]
init_u_data = np.array(init_nlpsolver_nlp_bin["data"]["u_data"])[:-1, :]
rel_u_data = np.array(nlpsolver_rel["data"]["u_data"])[:-1, :]

ax[0][1].plot(time_grid, best_x_data[:,6], color = "#d62728ff", label = "$T_\mathsf{fpsc}$")
ax[0][1].plot(time_grid, best_x_data[:,8], color = "#8c0e6ca0", label = "$T_\mathsf{vtsc}$")
ax[0][1].plot([time_grid[0], time_grid[-1]], [98.0, 98.0], color = "C7", linestyle="dashed", dashes=(2, 4))
ax[0][0].plot(time_grid, init_x_data[:,6], color = "#d62728ff", label = "$T_\mathsf{fpsc}$")
ax[0][0].plot(time_grid, init_x_data[:,8], color = "#8c0e6ca0", label = "$T_\mathsf{vtsc}$")
ax[0][0].plot([time_grid[0], time_grid[-1]], [98.0, 98.0], color = "C7", linestyle="dashed", dashes=(2, 4))

ax00a = ax[0][0].twinx()
ax00a.plot(time_grid, rel_c_data[:,1]/1e3, color = "darkorange", label = "$I_\mathsf{fpsc}$", linestyle='-.')
ax00a.plot(time_grid, rel_c_data[:,2]/1e3, color = "gold", label = "$I_\mathsf{vtsc}$", linestyle='-.')
ax00a.set_ylim(0,1)
ax00a.set_ylabel("Irrad. (kW/m²)")
ax01a = ax[0][1].twinx()
ax01a.plot(time_grid, rel_c_data[:,1]/1e3, color = "darkorange", label = "$I_\mathsf{fpsc}$", linestyle='-.')
ax01a.plot(time_grid, rel_c_data[:,2]/1e3, color = "gold", label = "$I_\mathsf{vtsc}$", linestyle='-.')
ax01a.set_ylim(0,1)
ax01a.set_ylabel("Irrad. (kW/m²)")

ax00a.legend(loc="upper right", framealpha=0.0)
ax01a.legend(loc="upper right", framealpha=0.0)

# l , h = ax[0][0].get_legend_handles_labels()
# l1, h1, = ax00a.get_legend_handles_labels()
# ax[0][0].legend(l+l1, h+h1, loc="lower center", framealpha=0.0, ncol=2)
# l , h = ax[0][1].get_legend_handles_labels()
# l1, h1, = ax01a.get_legend_handles_labels()
# ax[0][1].legend(l+l1, h+h1, loc="lower center", framealpha=0.0, ncol=2)

ax[0][1].plot(time_grid, rel_x_data[:,6], color = "#d62728ff", linestyle="dotted")
ax[0][1].plot(time_grid, rel_x_data[:,8], color = "#8c0e6ca0", linestyle="dotted")
ax[0][0].plot(time_grid, rel_x_data[:,6], color = "#d62728ff", linestyle="dotted")
ax[0][0].plot(time_grid, rel_x_data[:,8], color = "#8c0e6ca0", linestyle="dotted")

for ax_j in ax[0]:
    ax_j.set_ylim(0,105)
    # if ax_j == ax[0][0]:
    ax_j.set_ylabel("Temp. (°C)")
    ax_j.legend(loc="lower center", framealpha=0.0, bbox_to_anchor=(0.35,0))
    ax_j.get_yaxis().set_label_coords(-0.1,0.5)


ax[1][1].plot(time_grid, best_x_data[:,0], color="#d62728ff", label = "$T_\mathsf{ht,1}$")
ax[1][1].plot(time_grid, best_x_data[:,3], color="#d6272880", label = "$T_\mathsf{ht,4}$")
ax[1][1].plot([time_grid[0], time_grid[-1]], [98.0, 98.0], color = "C7", linestyle="dashed", dashes=(2, 4))
ax[1][0].plot(time_grid, init_x_data[:,0], color="#d62728ff", label = "$T_\mathsf{ht,1}$")
ax[1][0].plot(time_grid, init_x_data[:,3], color="#d6272880", label = "$T_\mathsf{ht,4}$")
ax[1][0].plot([time_grid[0], time_grid[-1]], [98.0, 98.0], color = "C7", linestyle="dashed", dashes=(2, 4))

ax[1][1].plot(time_grid, rel_x_data[:,0], color="#d62728ff", linestyle="dotted")
ax[1][1].plot(time_grid, rel_x_data[:,3], color="#d6272880", linestyle="dotted")
ax[1][0].plot(time_grid, rel_x_data[:,0], color="#d62728ff", linestyle="dotted")
ax[1][0].plot(time_grid, rel_x_data[:,3], color="#d6272880", linestyle="dotted")

for ax_j in ax[1]:
    ax_j.set_ylim(35,100)
    ax_j.set_ylabel("Temp. (°C)")
    ax_j.legend(loc="upper left", framealpha=0.0)
    ax_j.get_yaxis().set_label_coords(-0.1,0.5)

    
ax[2][1].plot(time_grid, best_x_data[:,4], color = "#1f77b4ff", label = "$T_\mathsf{lt}$")
ax[2][1].plot([time_grid[0], time_grid[-1]], [18.0, 18.0], color = "C7", linestyle="dashed", dashes=(2, 4))
ax[2][1].plot([time_grid[0], time_grid[-1]], [8.0, 8.0], color = "C7", linestyle="dashed", dashes=(2, 4))
ax[2][0].plot(time_grid, init_x_data[:,4], color = "#1f77b4ff", label = "$T_\mathsf{lt}$")
ax[2][0].plot([time_grid[0], time_grid[-1]], [18.0, 18.0], color = "C7", linestyle="dashed", dashes=(2, 4))
ax[2][0].plot([time_grid[0], time_grid[-1]], [8.0, 8.0], color = "C7", linestyle="dashed", dashes=(2, 4))

ax[2][1].plot(time_grid, rel_x_data[:,4], color = "#1f77b4ff", linestyle="dotted")
ax[2][0].plot(time_grid, rel_x_data[:,4], color = "#1f77b4ff", linestyle="dotted")

for ax_j in ax[2]:
    ax_j.set_ylim(5,25)
    ax_j.set_ylabel("Temp. (°C)", labelpad=12)
    ax_j.legend(loc="upper left", ncol=2, framealpha=0.0, bbox_to_anchor=(0, 1.05, 1.1, 0.0))
    ax_j.get_yaxis().set_label_coords(-0.1,0.5)


ax[3][1].step(time_grid[:-1], best_b_data[:,0], where="post", color = "#1f77b4ff", label = "$b_\mathsf{acm}$")
ax[3][1].step(time_grid[:-1], best_b_data[:,1], where="post", color = "#ff7f0eff", label = "$b_\mathsf{fc}$")
ax[3][1].step(time_grid[:-1], best_b_data[:,2], where="post", color = "C2", label = "$b_\mathsf{hp}$")
ax[3][0].step(time_grid[:-1], init_b_data[:,0], where="post", color = "#1f77b4ff", label = "$b_\mathsf{acm}$")
ax[3][0].step(time_grid[:-1], init_b_data[:,1], where="post", color = "#ff7f0eff", label = "$b_\mathsf{fc}$")
ax[3][0].step(time_grid[:-1], init_b_data[:,2], where="post", color = "C2", label = "$b_\mathsf{hp}$")

ax[3][1].step(time_grid[:-1], rel_b_data[:,0], color="#1f77b4ff", where="post", linestyle="dotted")
ax[3][1].step(time_grid[:-1], rel_b_data[:,1], color="#ff7f0eff", where="post", linestyle="dotted")
ax[3][1].step(time_grid[:-1], rel_b_data[:,2], color="C2", where="post", linestyle="dotted")
ax[3][0].step(time_grid[:-1], rel_b_data[:,0], color="#1f77b4ff", where="post", linestyle="dotted")
ax[3][0].step(time_grid[:-1], rel_b_data[:,1], color="#ff7f0eff", where="post", linestyle="dotted")
ax[3][0].step(time_grid[:-1], rel_b_data[:,2], color="C2", where="post", linestyle="dotted")

for ax_j in ax[3]:
    ax_j.set_ylim(0, 1.1)
    ax_j.set_ylabel(r"Status \{0, 1\}", labelpad=10)
    #ax_j.legend(loc="center left", framealpha=0.0, bbox_to_anchor=(0.08, 0.5, 1.1, 0.3))
    ax_j.legend(loc="upper right", framealpha=0.0, bbox_to_anchor=(1.06,1.1))
    # ax_j.legend(loc="upper right", framealpha=0.0)
    ax_j.get_yaxis().set_label_coords(-0.1,0.5)


ax[4][0].plot(time_grid, rel_c_data[:,0], color = "C0", label = "$T_\mathsf{amb}$", linestyle='-.')
ax[4][0].set_ylabel("Temp. (°C)", labelpad=12)
ax[4][0].set_ylim(10, 30)
ax40a = ax[4][0].twinx()
ax40a.plot(time_grid, rel_c_data[:,3]/1e3, color = "C1", label = "$\dot{Q}_\mathsf{lc}$", linestyle='-.')
ax40a.set_ylabel("Load (kW)")
ax40a.set_ylim(0, 9)
l , h = ax[4][0].get_legend_handles_labels()
l1, h1, = ax40a.get_legend_handles_labels()
ax[4][0].legend(l+l1, h+h1, loc="lower center", framealpha=0.0)

ax[4][1].legend(l+l1, h+h1, loc="lower center", framealpha=0.0)
ax[4][1].plot(time_grid, rel_c_data[:,0], color = "C0", label = "$T_\mathsf{amb}$", linestyle='-.')
ax[4][1].set_ylabel("Temp. (°C)", labelpad=12)
ax[4][1].set_ylim(10, 30)
ax41a = ax[4][1].twinx()
ax41a.plot(time_grid, rel_c_data[:,3]/1e3, color = "C1", label = "$\dot{Q}_\mathsf{lc}$", linestyle='-.')
ax41a.set_ylabel("Load (kW)")
ax41a.set_ylim(0, 9)
l , h = ax[4][1].get_legend_handles_labels()
l1, h1, = ax41a.get_legend_handles_labels()
ax[4][1].legend(l+l1, h+h1, loc="lower center", framealpha=0.0)
# ax[4][1].step(time_grid[:-1], best_u_data[:,0], where="post", color = "C0", label="$v_\mathsf{ppsc}$")
# ax[4][1].step(time_grid[:-1], best_u_data[:,2], where="post", color = "C1", label="$v_\mathsf{pssc}$")
# ax[4][1].step(time_grid[:-1], best_u_data[:,3], where="post", color = "C2", label="$P_\mathsf{g}$")
# ax[4][0].step(time_grid[:-1], init_u_data[:,0], where="post", color = "C0", label="$v_\mathsf{ppsc}$")
# ax[4][0].step(time_grid[:-1], init_u_data[:,2], where="post", color = "C1", label="$v_\mathsf{pssc}$")
# ax[4][0].step(time_grid[:-1], init_u_data[:,3], where="post", color = "C2", label="$P_\mathsf{g}$")

# ax[4][1].step(time_grid[:-1], rel_u_data[:,0], where="post", color = "C0", linestyle="dotted")
# ax[4][1].step(time_grid[:-1], rel_u_data[:,2], where="post", color = "C1", linestyle="dotted")
# ax[4][1].step(time_grid[:-1], rel_u_data[:,3], where="post", color = "C2", linestyle="dotted")
# ax[4][0].step(time_grid[:-1], rel_u_data[:,0], where="post", color = "C0", linestyle="dotted")
# ax[4][0].step(time_grid[:-1], rel_u_data[:,2], where="post", color = "C1", linestyle="dotted")
# ax[4][0].step(time_grid[:-1], rel_u_data[:,3], where="post", color = "C2", linestyle="dotted")


for ax_j in ax[4]:
    # ax_j.set_ylim(-0.1, 1.1)
    # ax_j.set_ylabel("Level [-1, 1]", labelpad=10)
    # ax_j.legend(loc="best", framealpha=0.0)
    # ax_j.legend(loc="upper right", framealpha=0.0)#, bbox_to_anchor=(1.06,1.1))
    ax_j.get_yaxis().set_label_coords(-0.1, 0.5)


for ax_j in ax[-1]:
    ax_j.set_xlim(time_grid[0], time_grid[-1])
    ax_j.set_xticks([0,4,8,12,16,20,24])

    # ax_j.set_xticks(time_grid)
    # ax_j.set_xticklabels(time_grid)
    ax_j.set_xlabel("Time (h)")

for ax_k in ax:
    for ax_k_j in ax_k:
        ax_k_j.spines["top"].set_visible(False)
        ax_k_j.spines["right"].set_visible(False)
for ax_k in [ax00a, ax01a, ax40a, ax41a]:
    ax_k.spines["top"].set_visible(False)
    # ax_k.spines["right"].set_visible(False)

# ax[0,0].set_title("Algorithm iteration: 0")
# ax[0,1].set_title("Algorithm iteration: 23 (Best)")

# plt.tight_layout()
plt.subplots_adjust(wspace=0.35)
# fig.savefig("latex/figures/energy_systems_traj.pdf", bbox_inches='tight', pad_inches=0.01)
# fig.savefig("../poster-elox/figures/energy_systems_traj.pdf", bbox_inches='tight', pad_inches=0.01)
plt.show();

In [ ]:
scaling = 1e1
energy_cost = rel_c_data[:,5]
grid_power_init = init_u_data[:,3] * scaling
grid_power_best = best_u_data[:,3] * scaling
cost_energy_init = grid_power_init * energy_cost[:-1] * 1e-2
cost_energy_best = grid_power_best * energy_cost[:-1] * 1e-2


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4, 4/golden))

ax.plot(time_grid[:-1], grid_power_init * energy_cost[:-1], label=r"$p_\mathrm{init}$")
ax.plot(time_grid[:-1], grid_power_best * energy_cost[:-1], label=r"$p_\mathrm{best}$")
ax.plot(time_grid[:-1], energy_cost[:-1], label=r"$p_g$", linestyle="-.")

ax.legend()
ax.set_ylabel(r"Energy cost ($\frac{\mathrm{ct}}{\mathrm{kWh}}\mathrm{kW}$)")
ax.set_xlabel(r"Time (h)")

In [ ]:
tmp = np.hstack([time_grid, np.array(24)])
step_grid = np.array([elm1 - elm for elm1, elm in zip(tmp[1:], tmp)])
step_grid

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.2, 1.2))

axa = ax.twinx()
axa.plot(time_grid[:-1], energy_cost[:-1] / 1e2, label=r"$p_\mathrm{g}$", linestyle="-.", color='tab:green')
axa.set_ylim(0,0.30)
axa.set_ylabel(r"Price (Eur/kWh)")

# ax.axhline(0, alpha=0.99, linestyle='dotted', color='black')
ax.plot(time_grid[:-1], np.cumsum(grid_power_init * energy_cost[:-1] / 1e2 * step_grid[:-1]), label=r"$c_\mathrm{first}$")
ax.plot(time_grid[:-1], np.cumsum(grid_power_best * energy_cost[:-1] / 1e2 * step_grid[:-1]), label=r"$c_\mathrm{best}$")
# ax.set_ylim(-0.5,8)
ax.set_ylabel(r"Cum. cost (Eur)")
ax.set_xlabel(r"Time (h)")
ax.grid(axis='y', linestyle='dotted')

l , h = ax.get_legend_handles_labels()
l1, h1, = axa.get_legend_handles_labels()
ax.legend(l+l1, h+h1, loc="upper left", framealpha=0.0)

for ax_k in [ax, axa]:
    ax_k.spines["top"].set_visible(False)
    ax_k.set_xlim(time_grid[0], time_grid[-1])
    ax_k.set_xticks([0,4,8,12,16,20,24])

fig.savefig("latex/figures/comparison_energy_consumption.pdf", bbox_inches='tight', pad_inches=0.01)

plt.show()

In [ ]:
raise

### Plot Voronoi cuts

In [ ]:
voronoi_dict = {}
for iter_nr in range(TOTAL_ITER):
    with open(RESULTS_DIR + MIQP_NAME(iter_nr=iter_nr), "rb") as f:
            nlpsolver_miqp = json.load(f)

    voronoi_dict[iter_nr] = {}
    try:
        voronoi_dict[iter_nr]['A_v'] = np.array(nlpsolver_miqp['data']['A_v'])
        voronoi_dict[iter_nr]['lb_v'] = np.array(nlpsolver_miqp['data']['lb_v'])
        voronoi_dict[iter_nr]['ub_v'] = np.array(nlpsolver_miqp['data']['ub_v'])
    except:
        voronoi_dict[iter_nr]['A_v'] = None
        voronoi_dict[iter_nr]['lb_v'] = None
        voronoi_dict[iter_nr]['ub_v'] = None

In [ ]:
iter_nr

In [ ]:
iter_nr = 4

In [ ]:
X = np.hstack((voronoi_dict[iter_nr]["A_v"], -voronoi_dict[iter_nr]["ub_v"][..., np.newaxis]))

In [ ]:
voronoi_dict[iter_nr]["A_v"].shape

In [ ]:
import pandas as pd
import seaborn as sns
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.manifold import TSNE

# We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne = TSNE(n_components, perplexity=2)
tsne_result = tsne.fit_transform(X)

# Plot the result of our TSNE with the label color coded
# A lot of the stuff here is about making the plot look pretty and not TSNE
tsne_result_df = pd.DataFrame(
    {'tsne_1': tsne_result[:,0],
    'tsne_2': tsne_result[:,1],
    'label': np.arange(0, tsne_result.shape[0])}
    )
fig, ax = plt.subplots(1)
my_palette = sns.color_palette("hls", tsne_result_df.shape[0])
sns.scatterplot(
    x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax,
    s=100, palette=my_palette, legend='full'
    )
lim = (tsne_result.min()-5, tsne_result.max()+5)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, ncol=2)

In [ ]:
# Generating a subplot with all the Voronoi iterations

fig = plt.figure(figsize=(12,12))
my_legend = False
my_palette = sns.color_palette("hls", voronoi_dict[TOTAL_ITER-1]['A_v'].shape[0])

for iter_nr in range(3, TOTAL_ITER):
    X = np.hstack(
        (voronoi_dict[iter_nr]["A_v"],
        -voronoi_dict[iter_nr]["ub_v"][..., np.newaxis])
        )
    # We want to get TSNE embedding with 2 dimensions
    n_components = 2
    tsne = TSNE(n_components, perplexity=X.shape[0]//2)
    tsne_result = tsne.fit_transform(X)

    # Plot the result of our TSNE with the label color coded
    # A lot of the stuff here is about making the plot look pretty and not TSNE
    tsne_result_df = pd.DataFrame(
        {'tsne_1': tsne_result[:,0],
        'tsne_2': tsne_result[:,1],
        'label': np.arange(0, tsne_result.shape[0])}
        )
    # fig, ax = plt.subplots(1)
    ax = fig.add_subplot(TOTAL_ITER//3+TOTAL_ITER%3, 3, iter_nr)
    if iter_nr==TOTAL_ITER-1:
        my_legend='full'
    sns.scatterplot(
        x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax,
        s=100, palette=my_palette[:tsne_result_df.shape[0]], legend=my_legend
        )
    lim = (tsne_result.min()-5, tsne_result.max()+5)
    ax.set_xlim(lim)
    ax.set_ylim(lim)
    ax.set_aspect('equal')


ax.get_legend().remove()
fig.legend(bbox_to_anchor=(1.05, 0.5), loc=2, borderaxespad=0.0, ncol=2)
plt.tight_layout()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Scaling the data:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca.fit(X_scaled)
pca_results = pca.transform(X_scaled)

print(f"Explained variance = {sum(pca.explained_variance_ratio_)}")
pca_results_df = pd.DataFrame(
    {'pca_1': pca_results[:,0],
    'pca_2': pca_results[:,1],
    'label': np.arange(0, pca_results.shape[0])}
)
print(pca_results_df.shape)

In [ ]:
fig, ax = plt.subplots(1)
my_palette = sns.color_palette("hls", pca_results_df.shape[0])

sns.scatterplot(
    x='pca_1', y='pca_2', hue='label', data=pca_results_df, ax=ax,
    s=100, palette=my_palette, legend='full'
    )
lim = (pca_results.min()-5, pca_results.max()+5)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, ncol=2)

In [ ]:
fig = plt.figure(figsize=(12,12))
my_legend = False
my_palette = sns.color_palette("hls", voronoi_dict[TOTAL_ITER-1]['A_v'].shape[0])

for iter_nr in range(2, TOTAL_ITER):
    X = np.hstack(
        (voronoi_dict[iter_nr]["A_v"],
        -voronoi_dict[iter_nr]["ub_v"][..., np.newaxis])
        )
    # Scaling
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    # PCA
    # pca = PCA(n_components=2, random_state=42)
    # pca.fit(X_scaled)
    pca_results = pca.transform(X_scaled)
    # Plot the result
    pca_results_df = pd.DataFrame(
        {'pca_1': pca_results[:,0],
        'pca_2': pca_results[:,1],
        'label': np.arange(0, pca_results.shape[0])}
        )

    ax = fig.add_subplot(TOTAL_ITER//3+TOTAL_ITER%3, 3, iter_nr)
    if iter_nr==TOTAL_ITER-1:
        my_legend='full'
    sns.scatterplot(
        x='pca_1', y='pca_2', hue='label', data=pca_results_df, ax=ax,
        s=100, palette=my_palette[:pca_results_df.shape[0]], legend=my_legend
        )
    lim = (pca_results.min()*1.1, pca_results.max()*1.1)
    ax.set_xlim(lim)
    ax.set_ylim(lim)
    ax.set_aspect('equal')


ax.get_legend().remove()
fig.legend(bbox_to_anchor=(1.05, 0.5), loc=2, borderaxespad=0.0, ncol=2)
plt.tight_layout()